In [1]:
!pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.4
    Uninstalling openai-1.54.4:
      Successfully uninstalled openai-1.54.4


In [2]:
!pip install torch transformers Pillow openai streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [3]:
!pip install pyngrok streamlit


In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import openai
import os
import base64

# OpenAI API key setup
openai.api_key = " "  # Replace with your actual OpenAI API key

# Load models for image captioning
caption_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
caption_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

# Function to generate captions for images
def create_image_captions(image):
    tensor_input = caption_processor(images=image, return_tensors="pt")
    generated_output = caption_model.generate(**tensor_input)
    caption_text = caption_processor.decode(generated_output[0], skip_special_tokens=True)
    return caption_text

# Function to compose a story from captions using GPT-3.5
def compose_story(captions_list):
    story_prompt = "In a land far away, "
    for caption in captions_list:
        story_prompt += f"{caption}. "

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a storytelling assistant."},
            {"role": "user", "content": story_prompt}
        ],
        max_tokens=300,
        temperature=0.7
    )
    generated_story = response.choices[0].message['content'].strip()
    return generated_story

# Function to apply custom CSS for background transparency and white text in input boxes
def set_background(image_file):
    st.markdown(
        f"""
        <style>
        .stApp {{
            background-image: url("data:image/webp;base64,{image_file}");
            background-size: cover;
            background-position: center;
            background-repeat: no-repeat;
            color: black;
            opacity: 0.8; /* Makes the background more transparent */
        }}
        h1, h2, h3, h4, h5, h6, p {{
            color: black !important;
        }}
        input, textarea, select {{
            color: white !important;  /* Makes text in inputs white */
            background-color: rgba(0, 0, 0, 0.7) !important;  /* Darkens input background for contrast */
        }}
        input::placeholder {{
            color: white !important; /* White placeholder text */
        }}
        .stTextInput label, .stButton button {{
            color: white !important; /* Makes labels and button text white */
        }}
        .stButton>button {{
            color: white !important; /* Ensures button text is white */
        }}
        .stAlert {{
            color: white !important; /* Makes alert messages like warnings appear in white */
            background-color: rgba(0, 0, 0, 0.6) !important;
        }}
        </style>
        """,
        unsafe_allow_html=True
    )

# Helper function to load and encode image for CSS
def load_image(image_path):
    with open(image_path, "rb") as image:
        return base64.b64encode(image.read()).decode()

# Load background images for pages
welcome_bg = load_image("/content/drive/MyDrive/LOGIN/DALL·E 2024-11-12 15.39.38 - A serene landscape in soft pastel colors, showing gentle hills, light greenery, and a soft, dreamy sky with delicate clouds. The scene is set in wide .webp")
signup_bg = load_image("/content/drive/MyDrive/LOGIN/DALL·E 2024-11-12 15.39.50 - A tranquil landscape in soft pastel colors, featuring rolling hills, light greenery, and a dreamy sky with wispy clouds. The scene is in wide format, .webp")
main_bg = load_image("/content/drive/MyDrive/LOGIN/DALL·E 2024-11-12 15.39.52 - A calming, wide-format landscape in soft pastel colors, with rolling hills, gentle greenery, and a serene sky filled with light clouds. Gentle sunligh.webp")


# Session state for page navigation
if "page" not in st.session_state:
    st.session_state.page = "welcome"

# Welcome Page
if st.session_state.page == "welcome":
    set_background(welcome_bg)
    st.title("Welcome to the Interactive AI Storyteller!")
    st.subheader("Unleash your imagination with the power of AI")
    st.write(
        """
        Embark on a creative journey with our AI-powered platform.
        Upload images, watch them transform into captivating captions,
        and let the magic of AI weave them into enchanting stories.
        Whether you're a writer, artist, or dreamer, this platform helps you unlock new creative dimensions.
        """
    )
    if st.button("Sign Up"):
        st.session_state.page = "signup"  # Navigate to Sign-Up Page

# Sign-Up Page
elif st.session_state.page == "signup":
    set_background(signup_bg)
    st.title("Sign Up")
    username = st.text_input("Enter your username")
    password = st.text_input("Enter your password", type="password")
    if st.button("Sign Up"):
        if username and password:
            st.session_state.page = "main"  # Redirect to main page
        else:
            st.warning("Please enter both username and password.")

# Main Interactive App Page
elif st.session_state.page == "main":
    set_background(main_bg)
    st.title("Interactive AI Storyteller")

    # Upload images
    uploaded_files = st.file_uploader("Upload images", accept_multiple_files=True, type=["png", "jpg", "jpeg", "webp"])

    captions = []
    images = []

    if uploaded_files:
        for uploaded_file in uploaded_files:
            image = Image.open(uploaded_file)
            caption = create_image_captions(image)
            captions.append(caption)
            images.append(image)

        st.write("Generated Captions:")
        for img, cap in zip(images, captions):
            st.image(img, caption=cap, use_column_width=True)

    # Compose story
    if captions:
        story = compose_story(captions)
        st.write("Generated Story:")
        st.write(story)

        feedback = st.text_input("Please provide feedback on the story:")
        if feedback:
            st.write("Thank you for your feedback!")

        add_more = st.button("Add More Images")
        if add_more:
            st.write("Please upload additional images to continue.")


Overwriting app.py


In [5]:
from pyngrok import ngrok

# Set up ngrok authentication
ngrok.set_auth_token("2oO1gv2ELmMn9OJ4RDxXMWinhEm_4mXLFQTcPqkAUk1VyhuHi")

In [6]:
import subprocess

# Run Streamlit in the background
streamlit_process = subprocess.Popen(['streamlit', 'run', 'app.py', '--server.port', '8501'])

# Create the public URL with ngrok
public_url = ngrok.connect(8501)
print(f"Public URL for Streamlit app: {public_url}")

Public URL for Streamlit app: NgrokTunnel: "https://24ed-35-231-123-139.ngrok-free.app" -> "http://localhost:8501"
